# Tubes a choc

In [ ]:
from trustutils import run

run.introduction('Y. GORSSE, M. NDJINGA')

### Description 
Issus du chapitre 10 du livre de Eleuterio F. Toro, Riemann Solvers and Numerical Methods for Fluid Dynamics, Third edition, Springer, 2009. 

Couramment utilisés comme benchmark pour le comportement de        méthodes numériques dans les ouvrages et publications scientifiques, ainsi que pour la vérification du comportement de certains codes.

Objectifs : montrer la la robutesse du code et évaluer 

1. l'apparition d'oscillations parasites, 
2. la propagation des ondes avec la bonne vitesse, 
3. la capture des états intermédiaires à la bonne hauteur.

In [ ]:
run.TRUST_parameters()

In [ ]:
######################################################################################################################
# 	This file contains a class to solve for the exact solution of the Riemann Problem for the one dimensional Euler
# 	equations with stiffened gas equation of state
#
# 	Author: Michael Ndjinga
# 	Date:	18/02/2021
#   Description : Translated from C++ package developped by Murray Cutforth
#######################################################################################################################

from math import pow, fabs, sqrt


class exact_rs_stiffenedgas:
    def __init__(self, gamma_L, gamma_R, pinf_L, pinf_R, tol=1.0e-6, max_iter=100):
        self.TOL = tol
        self.MAX_NB_ITER = max_iter

        self.gamma_L = gamma_L
        self.gamma_R = gamma_R
        self.pinf_L = pinf_L
        self.pinf_R = pinf_R

        self.S_STAR = 0.0
        self.P_STAR = 0.0
        self.rho_star_L = 0.0
        self.rho_star_R = 0.0

        self.S_L = 0.0
        self.S_R = 0.0
        self.S_HL = 0.0
        self.S_TL = 0.0
        self.S_HR = 0.0
        self.S_TR = 0.0

    # Functions used to generate exact solutions to Riemann problems

    def solve_RP(self, W_L, W_R):
        assert len(W_L) == 3, "Left state should have three components (rho, u p)"
        assert len(W_R) == 3, "Right state should have three components (rho, u p)"
        assert W_L[0] >= 0.0, "Left density should be positive"
        assert W_R[0] >= 0.0, "Right density should be positive"
        # assert W_L[2] >= 0.0 # Since stiffened gases will often exhibit p<0..
        # assert W_R[2] >= 0.0 #

        #print("")
        #print("Solving Riemann problem for left state W_L=", W_L, ", and right state W_R=", W_R)

        # Calculate p_star

        self.P_STAR = self.find_p_star_newtonraphson(W_L[0], W_L[1], W_L[2], W_R[0], W_R[1], W_R[2])

        # Calculate u_star

        self.S_STAR = 0.5 * (W_L[1] + W_R[1]) + 0.5 * (
            self.f(self.P_STAR, W_R[0], W_R[2], self.gamma_R, self.pinf_R)
            - self.f(self.P_STAR, W_L[0], W_L[2], self.gamma_L, self.pinf_L)
        )

        # Solution now depends on character of 1st and 3rd waves

        if self.P_STAR > W_L[2]:
            # Left shock

            self.rho_star_L = W_L[0] * (
                (2.0 * self.gamma_L * self.pinf_L + (self.gamma_L + 1.0) * self.P_STAR + (self.gamma_L - 1.0) * W_L[2])
                / (
                    2.0 * (W_L[2] + self.gamma_L * self.pinf_L)
                    + (self.gamma_L - 1.0) * self.P_STAR
                    + (self.gamma_L - 1.0) * W_L[2]
                )
            )
            self.S_L = W_L[1] - (self.Q_K(self.P_STAR, W_L[0], W_L[2], self.gamma_L, self.pinf_L) / W_L[0])
        else:
            # Left rarefaction

            self.rho_star_L = W_L[0] * pow((self.P_STAR + self.pinf_L) / (W_L[2] + self.pinf_L), 1.0 / self.gamma_L)

            a_L = self.a(W_L[0], W_L[2], self.gamma_L, self.pinf_L)
            a_star_L = a_L * pow(
                (self.P_STAR + self.pinf_L) / (W_L[2] + self.pinf_L), (self.gamma_L - 1.0) / (2.0 * self.gamma_L)
            )

            self.S_HL = W_L[1] - a_L
            self.S_TL = self.S_STAR - a_star_L

        if self.P_STAR > W_R[2]:
            # Right shock

            self.rho_star_R = W_R[0] * (
                (2.0 * self.gamma_R * self.pinf_R + (self.gamma_R + 1.0) * self.P_STAR + (self.gamma_R - 1.0) * W_R[2])
                / (
                    2.0 * (W_R[2] + self.gamma_R * self.pinf_R)
                    + (self.gamma_R - 1.0) * self.P_STAR
                    + (self.gamma_R - 1.0) * W_R[2]
                )
            )

            self.S_R = W_R[1] + (self.Q_K(self.P_STAR, W_R[0], W_R[2], self.gamma_R, self.pinf_R) / W_R[0])
        else:
            # Right rarefaction

            self.rho_star_R = W_R[0] * pow((self.P_STAR + self.pinf_R) / (W_R[2] + self.pinf_R), 1.0 / self.gamma_R)

            a_R = self.a(W_R[0], W_R[2], self.gamma_R, self.pinf_R)
            a_star_R = a_R * pow(
                (self.P_STAR + self.pinf_R) / (W_R[2] + self.pinf_R), (self.gamma_R - 1.0) / (2.0 * self.gamma_R)
            )

            self.S_HR = W_R[1] + a_R
            self.S_TR = self.S_STAR + a_star_R

    def sample_solution(self, W_L, W_R, S):
        W = [0.0] * 3

        # Find appropriate part of solution and return primitives

        if S < self.S_STAR:
            # To the left of the contact

            if self.P_STAR > W_L[2]:
                # Left shock

                if S < self.S_L:
                    W = W_L
                else:
                    W[0] = self.rho_star_L
                    W[1] = self.S_STAR
                    W[2] = self.P_STAR
            else:
                # Left rarefaction

                if S < self.S_HL:
                    W = W_L
                else:
                    if S > self.S_TL:
                        W[0] = self.rho_star_L
                        W[1] = self.S_STAR
                        W[2] = self.P_STAR
                    else:
                        self.set_left_rarefaction_fan_state(W_L, S, W)
        else:
            # To the right of the contact

            if self.P_STAR > W_R[2]:
                # Right shock

                if S > self.S_R:
                    W = W_R
                else:
                    W[0] = self.rho_star_R
                    W[1] = self.S_STAR
                    W[2] = self.P_STAR
            else:
                # Right rarefaction

                if S > self.S_HR:
                    W = W_R
                else:
                    if S < self.S_TR:
                        W[0] = self.rho_star_R
                        W[1] = self.S_STAR
                        W[2] = self.P_STAR
                    else:
                        self.set_right_rarefaction_fan_state(W_R, S, W)

        return W

    # Functions used to solve for p_star iteratively

    def find_p_star_newtonraphson(self, rho_L, u_L, p_L, rho_R, u_R, p_R):

        # First we set the initial guess for p_star using a simple mean-value approximation

        p_star_next = 0.5 * (p_L + p_R)
        n = 0

        # Now use the Newton-Raphson algorithm

        while True:  # conversion of do ... while by while True... if (...) break
            p_star = p_star_next

            p_star_next = p_star - self.total_pressure_function(
                p_star, rho_L, u_L, p_L, rho_R, u_R, p_R
            ) / self.total_pressure_function_deriv(p_star, rho_L, p_L, rho_R, p_R)

            p_star_next = max(p_star_next, self.TOL)

            n += 1

            if not ((fabs(p_star_next - p_star) / (0.5 * (p_star + p_star_next)) > self.TOL) and n < self.MAX_NB_ITER):
                break

        if n == self.MAX_NB_ITER:
            raise ValueError(
                "!!!!!!!!!!Newton algorithm did not converge. Increase tolerance or maximum number of time steps. Current values : tol="
                + str(self.TOL)
                + ", max_iter="
                + str(self.MAX_NB_ITER)
            )
            # p_star_next = 0.5*(p_L+p_R)

        return p_star_next

    def total_pressure_function(self, p_star, rho_L, u_L, p_L, rho_R, u_R, p_R):

        return (
            self.f(p_star, rho_L, p_L, self.gamma_L, self.pinf_L)
            + self.f(p_star, rho_R, p_R, self.gamma_R, self.pinf_R)
            + u_R
            - u_L
        )

    def total_pressure_function_deriv(self, p_star, rho_L, p_L, rho_R, p_R):

        return self.f_deriv(p_star, rho_L, p_L, self.gamma_L, self.pinf_L) + self.f_deriv(
            p_star, rho_R, p_R, self.gamma_R, self.pinf_R
        )

    def f(self, p_star, rho, p, gamma, pinf):
        if p_star > p:

            return (p_star - p) / self.Q_K(p_star, rho, p, gamma, pinf)

        else:

            return (2.0 * self.a(rho, p, gamma, pinf) / (gamma - 1.0)) * (
                pow((p_star + pinf) / (p + pinf), (gamma - 1.0) / (2.0 * gamma)) - 1.0
            )

    def f_deriv(self, p_star, rho, p, gamma, pinf):
        A = 2.0 / ((gamma + 1.0) * rho)
        B = (p + pinf) * (gamma - 1.0) / (gamma + 1.0)

        if p_star > p:

            return sqrt(A / (B + p_star + pinf)) * (1.0 - ((p_star - p) / (2.0 * (B + p_star + pinf))))

        else:

            return (1.0 / (rho * self.a(rho, p, gamma, pinf))) * pow(
                (p_star + pinf) / (p + pinf), -(gamma + 1.0) / (2.0 * gamma)
            )

    # Functions to find the state inside a rarefaction fan

    def set_left_rarefaction_fan_state(self, W_L, S, W):
        a_L = self.a(W_L[0], W_L[2], self.gamma_L, self.pinf_L)
        W[0] = W_L[0] * pow(
            (2.0 / (self.gamma_L + 1.0)) + ((self.gamma_L - 1.0) / (a_L * (self.gamma_L + 1.0))) * (W_L[1] - S),
            2.0 / (self.gamma_L - 1.0),
        )
        W[1] = (2.0 / (self.gamma_L + 1.0)) * (a_L + S + ((self.gamma_L - 1.0) / 2.0) * W_L[1])
        W[2] = (W_L[2] + self.pinf_L) * pow(
            (2.0 / (self.gamma_L + 1.0)) + ((self.gamma_L - 1.0) / (a_L * (self.gamma_L + 1.0))) * (W_L[1] - S),
            (2.0 * self.gamma_L) / (self.gamma_L - 1.0),
        ) - self.pinf_L

    def set_right_rarefaction_fan_state(self, W_R, S, W):
        a_R = self.a(W_R[0], W_R[2], self.gamma_R, self.pinf_R)
        W[0] = W_R[0] * pow(
            (2.0 / (self.gamma_R + 1.0)) - ((self.gamma_R - 1.0) / (a_R * (self.gamma_R + 1.0))) * (W_R[1] - S),
            2.0 / (self.gamma_R - 1.0),
        )
        W[1] = (2.0 / (self.gamma_R + 1.0)) * (-a_R + S + ((self.gamma_R - 1.0) / 2.0) * W_R[1])
        W[2] = (W_R[2] + self.pinf_R) * pow(
            (2.0 / (self.gamma_R + 1.0)) - ((self.gamma_R - 1.0) / (a_R * (self.gamma_R + 1.0))) * (W_R[1] - S),
            (2.0 * self.gamma_R) / (self.gamma_R - 1.0),
        ) - self.pinf_R

    # Misc functions

    def Q_K(self, p_star, rho, p, gamma, pinf):
        A = 2.0 / ((gamma + 1.0) * rho)
        B = (p + pinf) * (gamma - 1.0) / (gamma + 1.0)
        return sqrt((p_star + pinf + B) / A)

    # Equation of state functions

    def a(self, rho, p, gamma, pinf):  # sound speed
        return sqrt(gamma * ((p + pinf) / rho))


In [ ]:
from trustutils import plot

######################################################################################################################
#   This file runs a set of Riemann problems for the one dimensional Euler equations with stiffened gas equation of state.
#   It uses the class exact_rs_stiffenedgas to compute the exact solution of the Riemann Problems.
#
#   Author: Michael Ndjinga
#   Date:   19/02/2021
#   Description : Translated from C++ package developped by Murray Cutforth
#######################################################################################################################

from math import fabs
import os


def stiffenedgas_e(rho, p, gamma, pinf):
    return (p + gamma * pinf) / (rho * (gamma - 1))


def stiffenedgas_h(rho, p, gamma, pinf):
    return gamma * (p + pinf) / (rho * (gamma - 1))


def run_Riemann_problem(target_folder, TC, name, numsamples=100):
    # Output test solution for many different Riemann problems

    #print("")
    #print(
    #    "Determination of the exact solution of a Riemann problem for the Euler equations on "
    #    + str(numsamples)
    #    + " sample points."
    #)

    xmin = 0.0
    xmax = 1.0

    RS = exact_rs_stiffenedgas(TC["gamma"], TC["gamma"], TC["pinf"], TC["pinf"])
    RS.solve_RP(TC["WL"], TC["WR"])

    #print("")
    #print("Solved Riemann problem for TC = ", name)
    #print("Star state pressure calculated as ", RS.P_STAR)
    #print("Star state velocity calculated as ", RS.S_STAR)
    #print("Left star state density calculated as ", RS.rho_star_L)
    #print("Right state state density calculated as ", RS.rho_star_R)
    #print("Left shock speed calculated as ", RS.S_L)
    #print("Right shock speed calculated as ", RS.S_R)
    #print("Left rarefaction head speed calculated as ", RS.S_HL)
    #print("Left rarefaction tail speed calculated as ", RS.S_TL)
    #print("Right rarefaction head speed calculated as ", RS.S_HR)
    #print("Right rarefaction tail speed calculated as ", RS.S_TR)

    delx = (xmax - xmin) / numsamples

    outfile = open(f"{target_folder}/{name}.ex", "w")
    outfile.write("x               RHO                 V                 P              EINT\n")

    t = TC["tmax"]
    offset = TC["x"]
    x = xmin
    maxc = 0
    while x <= xmax:
        S = x / t
        soln = RS.sample_solution(TC["WL"], TC["WR"], S - offset / t)
        thisgamma = TC["gamma"]
        thispinf = TC["pinf"]
        thisz = 1.0 if S - offset / t < RS.S_STAR else 0.0
        outfile.write(
            str(x)
            + " "
            + str(soln[0])
            + " "
            + str(soln[1])
            + " "
            + str(soln[2])
            + " "
            + str(stiffenedgas_e(soln[0], soln[2], thisgamma, thispinf))
            + " "
            + str(stiffenedgas_h(soln[0], soln[2], thisgamma, thispinf))
            + " "
            + str(fabs(soln[1]) / RS.a(soln[0], soln[2], thisgamma, thispinf))
            + " "
            + str(thisz)
            + "\n"
        )
        maxc = max(maxc, fabs(soln[1] + RS.a(soln[0], soln[2], thisgamma, thispinf)))
        x += delx
    outfile.close()
    return maxc


def generate_cases(target_folder):
    TCs = {
        "Toro1": {"WL": [1, 0.75, 1], "WR": [0.125, 0, 0.1], "tmax": 0.2, "gamma": 1.4, "pinf": 0, "x": 0.3},
        "Toro2": {"WL": [1, -2, 0.4], "WR": [1, 2, 0.4], "tmax": 0.15, "gamma": 1.4, "pinf": 0, "x": 0.5},
        "Toro3": {"WL": [1, 0, 1000], "WR": [1, 0, 0.01], "tmax": 0.012, "gamma": 1.4, "pinf": 0, "x": 0.5},
        "Toro4": {
            "WL": [5.99924, 19.5975, 460.894],
            "WR": [5.99242, -6.19633, 46.0950],
            "tmax": 0.035,
            "gamma": 1.4,
            "pinf": 0,
            "x": 0.4,
        },
        "Toro5": {
            "WL": [1, -19.5975, 1000],
            "WR": [1, -19.59745, 0.01],
            "tmax": 0.012,
            "gamma": 1.4,
            "pinf": 0,
            "x": 0.8,
        },
        "Toro6": {"WL": [1.4, 0, 1], "WR": [1.4, 0, 1], "tmax": 2, "gamma": 1.4, "pinf": 0, "x": 0.5},
        "Toro7": {"WL": [1.4, 0.1, 1], "WR": [1, 0.1, 1], "tmax": 2, "gamma": 1.4, "pinf": 0, "x": 0.5},
        "PWR1": {
            "WL": [700, 0, 155e5],
            "WR": [700, 0, 1e5],
            "tmax": 3e-4,
            "gamma": 1.58,
            "pinf": 353637173.0,
            "x": 0.5,
        },
        "PWR2": {
            "WL": [700, 0, 155e5],
            "WR": [700, 20, 155e5],
            "tmax": 3e-4,
            "gamma": 1.58,
            "pinf": 353637173.0,
            "x": 0.5,
        },
        "PWR3": {
            "WL": [700, 0, 155e5],
            "WR": [650, 0, 155e5],
            "tmax": 3e-4,
            "gamma": 1.58,
            "pinf": 353637173.0,
            "x": 0.5,
        },
    }
    meshes = [100, 200, 400, 800]
    for n in meshes:
        for d in ["VDF", "PolyMAC_P0"]:
            os.system(f"mkdir -p {target_folder}/n{n}/{d}")

    columns = ["$$\gamma$$", "$$p_\infty$$", "$$\rho_L$$", "$$u_L$$", "$$p_L$$", "$$\rho_R$$", "$$u_R$$", "$$p_R$$","$$x_d$$"]
    tab = plot.Table(columns)
    for name in ["Toro1", "Toro2", "Toro3", "Toro4", "Toro5", "Toro6", "Toro7", "PWR1", "PWR2", "PWR3"]:

        with open(f"{target_folder}/jdd.data", "r") as file:
            filedata = file.read()

        # Replace the target string
        for i, n in enumerate(["__rl__", "__vl__", "__pl__"]):
            filedata = filedata.replace(n, str(TCs[name]["WL"][i]))
        for i, n in enumerate(["__rr__", "__vr__", "__pr__"]):
            filedata = filedata.replace(n, str(TCs[name]["WR"][i]))
        filedata = filedata.replace(
            "__Tl__", str((TCs[name]["WL"][2] + TCs[name]["pinf"]) / (TCs[name]["WL"][0] * 8.31446261815324) -273.15 )
        )
        filedata = filedata.replace(
            "__Tr__", str((TCs[name]["WR"][2] + TCs[name]["pinf"]) / (TCs[name]["WR"][0] * 8.31446261815324) -273.15 )
        )

        tab.addLine( [[TCs[name]["gamma"], TCs[name]["pinf"]] + TCs[name]["WL"] + TCs[name]["WR"] + [TCs[name]["x"]]], name)

        for n in ["tmax", "gamma", "pinf", "x"]:
            filedata = filedata.replace("__{}__".format(n), str(TCs[name][n]))

        c = run_Riemann_problem(target_folder, TCs[name], name, 1000)

        # Write the file out again
        for n in meshes:
            dt = TCs[name]["tmax"] / n
            # dt = 0.1 * 1.0 / n / c
            filedata_ = filedata.replace("__n__", str(n + 1))
            filedata_ = filedata_.replace("__dt__", str(dt))
            for d, d_str in [("VDF", "VDF dis Option_VDF { p_imposee_aux_faces oui traitement_coins oui }"), ("PolyMAC_P0", "PolyMAC_P0 dis Option_PolyMAC_P0 {  }")]:
                with open(f"{target_folder}/n{n}/{d}/{name}.data", "w") as file:
                    file.write(filedata_.replace("__dis__", d_str))
    return tab

In [ ]:
run.reset()
run.initBuildDirectory()

tab = generate_cases(run.BUILD_DIRECTORY)

list_meshes = [100, 800]
list_tests = ['Toro1','Toro2','Toro4','Toro5','Toro6','Toro7','PWR1','PWR2','PWR3']

for m in list_meshes:
    for dis in ["PolyMAC_P0", "VDF"]:
        for test in list_tests:
            run.addCase(f"n{m}/{dis}", f"{test}.data")

run.printCases()
run.runCases()

## Description des cas tests

These tests are designed to assess the robustness and accuracy of numerical methods at the core of the solver, independently from the boundary condition, and source term treatment involving correla       tions.

They consist in the numerical resolution of the Shock tube problem for a perfect gas with $\gamma=1.4$. 

The initial state consists in two constant states $W_L=(\rho_L,u_L,p_L)$ and $W_R=(\rho_R,u_R,p_R)$ separated by a discontinuity at $x=x_d$. The following table gives the values of $W_L$ and $W_R$ for each test.

In [ ]:
display(tab)

In [ ]:
variables = ["RHO", "P", "V", "EINT"]

dis_m = {"VDF" : "x-", "PolyMAC_P0" : "-"}

for test in list_tests:
    columns = plot.readFile(f"{run.BUILD_DIRECTORY}/{test}.ex")
    var = columns[0].split()
    data = plot.loadText(f"{run.BUILD_DIRECTORY}/{test}.ex", skiprows=1)
    
    a = plot.Graph(nX=2, nY=2, title=test)
    nX, nY = 0, 0
    for p in variables:
        a.addPlot([nX, nY], f"Profil de {p}")
        for m in list_meshes:
            for dis, ma in dis_m.items():
                a.addSegment(f"n{m}/{dis}/{test}_{p}.son", label=f"{dis} n={m}",lw = 4, marker=ma)
        nb = var.index(p)
        a.add(data[0], data[nb], label="Exact solution", lw=2, color='k')
        a.label("z [m]", p)
        
        nX = nX+1 if nY%2 else nX
        nY = (nY+1)%2